In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
companies_ratio_df = pd.read_excel('data_to_prepare/companies_ratios.xlsx')
companies_list_df = pd.read_excel('data_to_prepare/companies_list.xlsx')[['Name', 'permno']]

# drop duplicates of companies_ratio_df on 'qdate' and 'permno'
companies_ratio_df = companies_ratio_df.drop_duplicates(subset=['permno', 'qdate'])

companies_stock_df = pd.read_excel('data_to_prepare/companies_stock.xlsx')[['permno', 'date', 'PRC', 'RET']]
# Convert to numeric and drop the rows with missing values
companies_stock_df['PRC'] = pd.to_numeric(companies_stock_df['PRC'], errors='coerce')
companies_stock_df['RET'] = pd.to_numeric(companies_stock_df['RET'], errors='coerce')

# Interpolate the stock price for each company using np.interp
for permno in companies_ratio_df['permno'].unique():
    ratio_df = companies_ratio_df[companies_ratio_df['permno'] == permno]
    stock_df = companies_stock_df[companies_stock_df['permno'] == permno]
    x = stock_df['date'].astype(int)
    y1 = stock_df['PRC']
    y2 = stock_df['RET']
    companies_ratio_df.loc[companies_ratio_df['permno'] == permno, 'PRC'] = np.interp(ratio_df['qdate'].astype(int), x, y1)
    companies_ratio_df.loc[companies_ratio_df['permno'] == permno, 'RET'] = np.interp(ratio_df['qdate'].astype(int), x, y2)
# Backfill the stock price and return for each company
companies_ratio_df['PRC'] = companies_ratio_df.groupby('permno')['PRC'].bfill().ffill()
companies_ratio_df['RET'] = companies_ratio_df.groupby('permno')['RET'].bfill().ffill()

# Add macro variables
hpi = pd.read_excel('data_to_prepare/S&P_HPI.xlsx')
# Interpolate the hpi for each company using np.interp
for permno in companies_ratio_df['permno'].unique():
    ratio_df = companies_ratio_df[companies_ratio_df['permno'] == permno]
    x = hpi['DATE'].astype(int)
    y = hpi['hpi']
    companies_ratio_df.loc[companies_ratio_df['permno'] == permno, 'HPI'] = np.interp(ratio_df['qdate'].astype(int), x, y)
# Backfill the hpi for each company
companies_ratio_df['HPI'] = companies_ratio_df.groupby('permno')['HPI'].bfill().ffill()

ted_rate = pd.read_excel('data_to_prepare/TEDRATE.xlsx')
# Convert to numeric and drop the rows with missing values
ted_rate['ted_rate'] = pd.to_numeric(ted_rate['ted_rate'], errors='coerce')
# Interpolate the ted rate for each company using np.interp
for permno in companies_ratio_df['permno'].unique():
    ratio_df = companies_ratio_df[companies_ratio_df['permno'] == permno]
    x = ted_rate['DATE'].astype(int)
    y = ted_rate['ted_rate']
    companies_ratio_df.loc[companies_ratio_df['permno'] == permno, 'TEDSPREAD'] = np.interp(ratio_df['qdate'].astype(int), x, y)
# Backfill the ted rate for each company
companies_ratio_df['TEDSPREAD'] = companies_ratio_df.groupby('permno')['TEDSPREAD'].bfill().ffill()

un_rate = pd.read_excel('data_to_prepare/UNRATE.xlsx')
# Interpolate the un rate for each company using np.interp
for permno in companies_ratio_df['permno'].unique():
    ratio_df = companies_ratio_df[companies_ratio_df['permno'] == permno]
    x = un_rate['DATE'].astype(int)
    y = un_rate['unrate']
    companies_ratio_df.loc[companies_ratio_df['permno'] == permno, 'UNRATE'] = np.interp(ratio_df['qdate'].astype(int), x, y)
# Backfill the un rate for each company
companies_ratio_df['UNRATE'] = companies_ratio_df.groupby('permno')['UNRATE'].bfill().ffill()

delinquency_rate = pd.read_excel('data_to_prepare/DRSFRMACBS.xlsx')
# Interpolate the delinquency rate for each company using np.interp
for permno in companies_ratio_df['permno'].unique():
    ratio_df = companies_ratio_df[companies_ratio_df['permno'] == permno]
    x = delinquency_rate['DATE'].astype(int)
    y = delinquency_rate['delinq_rate']
    companies_ratio_df.loc[companies_ratio_df['permno'] == permno, 'DELINQRATE'] = np.interp(ratio_df['qdate'].astype(int), x, y)

# Normalize the macro variables
companies_ratio_df['HPI'] = (companies_ratio_df['HPI'] - hpi['hpi'].mean()) / hpi['hpi'].std()
companies_ratio_df['TEDSPREAD'] = (companies_ratio_df['TEDSPREAD'] - ted_rate['ted_rate'].mean()) / ted_rate['ted_rate'].std()
companies_ratio_df['UNRATE'] = (companies_ratio_df['UNRATE'] - un_rate['unrate'].mean()) / un_rate['unrate'].std()
companies_ratio_df['DELINQRATE'] = (companies_ratio_df['DELINQRATE'] - delinquency_rate['delinq_rate'].mean()) / delinquency_rate['delinq_rate'].std()

# Merge companies_ratio_df with companies_list_df on 'permno'
companies_ratio_df = companies_ratio_df.merge(companies_list_df, on='permno', how='left')

# Converting to long format
df = companies_ratio_df
ref_date = pd.to_datetime('1998-01-01')
# Start and stop time for each company
for Name in df['Name'].unique():
    df.loc[df['Name'] == Name, 'start'] = (df.loc[df['Name'] == Name, 'qdate'].shift(1) - ref_date).dt.days
    df.loc[df['Name'] == Name, 'stop'] = (df.loc[df['Name'] == Name, 'qdate'] - ref_date).dt.days
df.loc[df['start'].isna(), 'start'] = df.loc[df['start'].isna(), 'stop'] - 90
df['start'] = df['start'].astype(int)
df['stop'] = df['stop'].astype(int)
df.loc[:, 'start'] = df['start'] + 90
df.loc[:, 'stop'] = df['stop'] + 90

# # Add first difference of covariates
cova_for_diff = ['cash_lt', 'debt_at', 'short_debt', 'lt_debt', 'cash_debt',
       'debt_assets', 'debt_capital', 'de_ratio', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'PRC', 'RET', 'HPI',
       'TEDSPREAD', 'UNRATE', 'DELINQRATE']
for cova in cova_for_diff:
    df['d' + cova] = df.groupby('Name')[cova].diff()



# # Compute the cumulative returns for each company
companies_ratio_df['cum_ret'] = (1 + companies_ratio_df['RET']).groupby(companies_ratio_df['Name']).cumprod()
# Compute the running maximum of the cumulative returns
companies_ratio_df['running_max'] = companies_ratio_df.groupby('Name')['cum_ret'].cummax()
# Compute the drawdown for each company
companies_ratio_df['drawdown'] = companies_ratio_df['cum_ret'] / companies_ratio_df['running_max'] - 1
# Compute the max drawdown for each company
companies_ratio_df['max_drawdown'] = companies_ratio_df.groupby('Name')['drawdown'].cummin()
# # If you want the maximum drawdown per company as a summary
# max_drawdown_per_company = companies_ratio_df.groupby('Name')['drawdown'].min()
# Delete the columns that are not needed anymore
companies_ratio_df = companies_ratio_df.drop(columns=['cum_ret', 'running_max', 'drawdown'])

# Create a new column 'distress' initialized with 0
companies_ratio_df['distress'] = 0
companies_list_df = pd.read_excel('data_to_prepare/companies_list.xlsx')[['permno', 'distress']]
# Filter for companies marked as distressed in companies_list
distressed_companies = companies_list_df[companies_list_df['distress'] == 1]

# Loop through each distressed company and mark the day of maximum drawdown as distress (1)
for permno in distressed_companies['permno']:
    # Filter rows for the current company in companies_ratio_df
    company_data = companies_ratio_df[companies_ratio_df['permno'] == permno]
    
    # Find the day with the maximum drawdown
    if not company_data.empty:
        max_drawdown_day = company_data['max_drawdown'].idxmin()
        
        # Set 'distress' to 1 on the day of maximum drawdown
        companies_ratio_df.at[max_drawdown_day, 'distress'] = 1
# For each company in distress, delete the data after the bankruptcy date
for Name in companies_ratio_df['Name'].unique():
    if companies_ratio_df[companies_ratio_df['Name'] == Name]['distress'].sum() > 0:
        bankrupt_date = companies_ratio_df[(companies_ratio_df['Name'] == Name) & (companies_ratio_df['distress'] == 1)]['stop'].values[0]
        companies_ratio_df = companies_ratio_df[(companies_ratio_df['Name'] != Name) | (companies_ratio_df['stop'] <= bankrupt_date)]

# Drop useless columns
companies_ratio_df = companies_ratio_df.drop(columns=['permno', 'adate', 'public_date'])
# Save the data
companies_ratio_df.to_excel('clean_data/companies_ratio_final.xlsx', index=False)
companies_ratio_df

,qdate,cash_lt,debt_at,short_debt,lt_debt,cash_debt,debt_assets,debt_capital,de_ratio,inv_turn,...,dsale_invcap,dsale_equity,dPRC,dRET,dHPI,dTEDSPREAD,dUNRATE,dDELINQRATE,max_drawdown,distress
0,1999-09-30,0.049,0.276,0.328,0.203,-0.083,0.913,0.912,10.492,252.801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0
1,1999-12-31,0.037,0.294,0.825,0.056,-0.077,0.910,0.909,10.124,170.376,...,0.294,-0.049,0.000000,0.000000,0.039667,-1.623053,-0.054474,-0.011856,-0.053548,0
2,2000-03-31,0.044,0.296,0.368,0.205,-0.077,0.911,0.909,10.175,198.106,...,-0.270,0.108,-2.312500,0.201935,0.085574,0.000000,-0.102032,0.008392,-0.053548,0
3,2000-06-30,0.045,0.292,0.335,0.213,-0.077,0.910,0.909,10.113,189.828,...,0.010,0.055,3.000000,-0.254066,0.108170,1.032852,0.102032,0.031322,-0.105679,0
4,2000-09-30,0.045,0.288,0.358,0.203,-0.077,0.909,0.908,9.992,207.602,...,0.034,0.057,13.621094,0.150119,0.068433,-0.625332,-0.052717,0.040096,-0.105679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2007-06-30,0.026,0.086,0.259,0.070,0.001,0.918,0.917,11.152,1.028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0
1507,2007-09-30,0.025,0.087,0.226,0.074,0.001,0.915,0.915,10.797,1.128,...,0.014,0.036,0.000000,0.000000,-0.116397,1.686289,0.001757,0.112053,-0.074017,0
1508,2007-12-31,0.009,0.096,0.415,0.062,0.000,0.911,0.910,10.193,1.340,...,-0.045,-0.176,-2.982606,-0.056149,-0.218361,-0.056210,0.158150,0.165890,-0.194548,0
1509,2008-03-31,0.025,0.093,0.307,0.071,0.000,0.917,0.916,11.030,1.496,...,0.030,0.139,-5.440000,-0.037942,-0.129263,-0.168629,0.001701,0.194852,-0.329951,0
